# Comparison of MORDM and MORO policies

## 8 Objective

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from ema_workbench import Policy
import functools


from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.util import ema_logging
from problem_formulation import get_model_for_problem_formulation

from ema_workbench.em_framework.evaluators import (BaseEvaluator,MultiprocessingEvaluator)
from ema_workbench.analysis import parcoords

ema_logging.log_to_stderr(ema_logging.INFO)
BaseEvaluator.reporting_frequency = 0.1
# ema_logging.log_to_stderr(ema_logging.DEBUG)

C:\AnacondaFiles\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\AnacondaFiles\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\AnacondaFiles\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
dike_model = get_model_for_problem_formulation(4)

[MainProcess/INFO] model initialized


In [3]:
from ema_workbench import (load_results, save_results)

results = load_results('open_exploration_10k_4pol_8obj.tar.gz')
experiments, outcomes = results
OUT_DF = pd.DataFrame(outcomes)
OUT_DF.describe()

[MainProcess/INFO] results loaded succesfully from C:\Users\Merih Karabulut\Desktop\epa1361_open-master\final assignment\open_exploration_10k_4pol_8obj.tar.gz


,Expected Evacuation Costs,Expected Number of Deaths,Gelderland Dike Investment Costs,GelderlandDownstream Expected Annual Damage,GelderlandUpstream Expected Annual Damage,OverIjsel Dike Investment Costs,OverIjsel Expected Annual Damage,RfR Total Costs
count,40000.000000,40000.000000,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04
mean,2.970466,0.232953,1.138756e+08,5.481896e+07,1.143741e+08,2.078718e+07,4.009794e+07,1.776000e+08
std,49.323626,0.416554,7.330628e+07,1.402370e+08,2.650632e+08,2.078744e+07,7.179330e+07,3.076161e+08
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000286,7.467223e+07,2.940871e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000,0.025747,1.351245e+08,6.884485e+06,1.313497e+06,2.078718e+07,4.490522e+06,0.000000e+00
75%,0.000000,0.247221,1.743279e+08,3.190093e+07,5.220290e+07,4.157436e+07,4.669935e+07,1.776000e+08
max,2318.932601,2.493580,1.852532e+08,8.703649e+08,1.505129e+09,4.157436e+07,5.122682e+08,7.104000e+08


In [4]:
def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]
    
def costs(data):
    return data[0]/1e9

def cost(data):
    return data[0]/1e4

SMALLER = 'SMALLER'

#values form the open exploration in the 50% of the cases + literature (National Water Plan 2016-2021)

expected_number_of_deaths = functools.partial(robustness, SMALLER, 0.001)
gel_downstream_expected_annual_damage = functools.partial(robustness, SMALLER, 6.884485e+06)
gel_upstream_expected_annual_damage = functools.partial(robustness, SMALLER, 1.313497e+06)
ijs_expected_annual_damage = functools.partial(robustness, SMALLER, 4.490522e+06)
evacuation_costs = cost
gel_dike_investment_costs = costs
ijs_dike_investment_costs = costs
rfr_total_investment_costs = costs

In [5]:
robustness_funcs = {'Expected Number of Deaths': expected_number_of_deaths,
                    'GelderlandDownstream Expected Annual Damage': gel_downstream_expected_annual_damage,
                    'GelderlandUpstream Expected Annual Damage': gel_upstream_expected_annual_damage,
                    'OverIjsel Expected Annual Damage': ijs_expected_annual_damage,
                    'Expected Evacuation Costs': evacuation_costs,                 
                    'Gelderland Dike Investment Costs': gel_dike_investment_costs,
                    'OverIjsel Dike Investment Costs': ijs_dike_investment_costs, 
                    'RfR Total Costs': rfr_total_investment_costs}

In [6]:
mordm_policies = pd.read_csv('mordm_optimal_8formul_pol.csv', index_col=0)

,A.1_DikeIncrease,A.2_DikeIncrease,A.3_DikeIncrease,A.4_DikeIncrease,A.5_DikeIncrease,0_RfR,1_RfR,2_RfR,3_RfR,4_RfR,EWS_DaysToThreat
0,10,7,10,10,10,0,0,0,1,0,3
1,9,7,10,9,10,0,0,0,1,0,3
2,9,9,10,6,10,0,0,0,1,0,4
3,9,8,10,6,10,0,0,0,1,0,3
4,10,8,10,5,10,0,0,0,1,0,3
5,9,9,10,5,10,0,0,0,1,0,3
6,9,8,10,5,10,0,0,0,1,0,3
7,9,9,10,4,10,0,0,0,1,0,3


In [7]:
policies_to_evaluate = []

for i, policy in mordm_policies.iterrows():
    policies_to_evaluate.append(Policy("mordm {}".format(i), **policy.to_dict()))

In [9]:
moro_policies = pd.read_csv('MORO_policies_8obj_500_pol.csv', index_col=0)
moro_policies
for i, policy in moro_policies.iterrows():
    policies_to_evaluate.append(Policy("moro_rfr_opt {}".format(i), **policy.to_dict()))

In [10]:
moro_policies = pd.read_csv('MORO_policies_8obj_filtered_pol.csv', index_col=0)
moro_policies
for i, policy in moro_policies.iterrows():
    policies_to_evaluate.append(Policy("moro_geldinv_opt {}".format(i), **policy.to_dict()))

In [11]:
POL_DF = pd.DataFrame(policies_to_evaluate)

,0_RfR,1_RfR,2_RfR,3_RfR,4_RfR,A.1_DikeIncrease,A.2_DikeIncrease,A.3_DikeIncrease,A.4_DikeIncrease,A.5_DikeIncrease,EWS_DaysToThreat
0,0,0,0,1,0,10,7,10,10,10,3
1,0,0,0,1,0,9,7,10,9,10,3
2,0,0,0,1,0,9,9,10,6,10,4
3,0,0,0,1,0,9,8,10,6,10,3
4,0,0,0,1,0,10,8,10,5,10,3
5,0,0,0,1,0,9,9,10,5,10,3
6,0,0,0,1,0,9,8,10,5,10,3
7,0,0,0,1,0,9,9,10,4,10,3
8,0,0,0,0,0,7,8,9,6,7,2
9,1,1,1,1,1,2,6,9,3,5,4


In [12]:
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    re_evaluation_results = evaluator.perform_experiments(n_scenarios,
                                                          policies_to_evaluate)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 100 scenarios * 36 policies * 1 model(s) = 3600 experiments
[MainProcess/INFO] 360 cases completed
[MainProcess/INFO] 720 cases completed
[MainProcess/INFO] 1080 cases completed
[MainProcess/INFO] 1440 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2160 cases completed
[MainProcess/INFO] 2520 cases completed
[MainProcess/INFO] 2880 cases completed
[MainProcess/INFO] 3240 cases completed
[MainProcess/INFO] 3600 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [44]:
experiments, outcomes = re_evaluation_results

overall_robustness = {}
for policy in np.unique(experiments['policy']):
    policy_robustness = {}

    logical = experiments['policy'] == policy
    
    for outcome, values in outcomes.items():
        values = values[logical]
        policy_robustness[outcome] = robustness_funcs[outcome](values)
    overall_robustness[policy] = policy_robustness
overall_robustness = pd.DataFrame.from_dict(overall_robustness).T

In [15]:
#overall_robustness = overall_robustness.sort_values(by = "Expected Number of Deaths", ascending = False)
overall_robustness.to_csv('comparison_8obj_final.csv')

In [52]:
overall_robustnessDF = pd.DataFrame(overall_robustness)
overall_robustnessDF = overall_robustnessDF.reset_index()

In [53]:
POL_DF = pd.DataFrame(policies_to_evaluate)

In [54]:
FinalDF = pd.concat([POL_DF,overall_robustnessDF], axis = 1)

In [55]:
FinalDF.sort_values(by = "Expected Number of Deaths", ascending = False)

,0_RfR,1_RfR,2_RfR,3_RfR,4_RfR,A.1_DikeIncrease,A.2_DikeIncrease,A.3_DikeIncrease,A.4_DikeIncrease,A.5_DikeIncrease,EWS_DaysToThreat,index,Expected Evacuation Costs,Expected Number of Deaths,Gelderland Dike Investment Costs,GelderlandDownstream Expected Annual Damage,GelderlandUpstream Expected Annual Damage,OverIjsel Dike Investment Costs,OverIjsel Expected Annual Damage,RfR Total Costs
2,0,0,0,1,0,9,9,10,6,10,4,mordm 2,0.000000,0.97,0.171407,0.98,0.95,0.041574,1.00,0.1212
3,0,0,0,1,0,9,8,10,6,10,3,mordm 3,0.000000,0.97,0.167020,0.98,0.93,0.041574,1.00,0.1212
4,0,0,0,1,0,10,8,10,5,10,3,mordm 4,0.000000,0.96,0.171040,0.96,0.93,0.041574,1.00,0.1212
5,0,0,0,1,0,9,9,10,5,10,3,mordm 5,0.000000,0.96,0.170476,0.96,0.95,0.041574,1.00,0.1212
6,0,0,0,1,0,9,8,10,5,10,3,mordm 6,0.000000,0.96,0.166088,0.96,0.93,0.041574,1.00,0.1212
33,1,1,1,1,1,1,6,10,2,7,3,moro_geldinv_opt 7,0.000000,0.96,0.148700,0.96,0.93,0.041574,0.99,0.3331
7,0,0,0,1,0,9,9,10,4,10,3,mordm 7,0.000000,0.95,0.169595,0.93,0.95,0.041574,1.00,0.1212
23,1,1,1,1,1,0,7,9,3,4,3,moro_geldinv_opt 4,0.000000,0.95,0.137254,0.99,0.87,0.041574,1.00,0.4543
0,0,0,0,1,0,10,7,10,10,10,3,mordm 0,0.000000,0.93,0.172080,0.99,0.90,0.041574,1.00,0.1212
1,0,0,0,1,0,9,7,10,9,10,3,mordm 1,0.000000,0.93,0.165970,0.99,0.90,0.041574,1.00,0.1212


In [57]:
FinalDF.to_csv('comparison_8obj_real_final.csv')